In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline


In [ ]:
print(__doc__)

import os as os
import numpy as np
import matplotlib.pyplot as plt
from scipy import io
from scipy.signal import stft, istft, spectrogram
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from numpy import linalg


In [ ]:
plt.rcParams['figure.figsize'] = (12.0, 9.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rc('text', usetex=True)
plt.rc('font', family='times')
plt.rcParams.update({'font.size': 26})

In [ ]:
figPath = './figs-01-16/'
fs = int(10e6)
nfft = 4096
nperseg = 1024
beta_loss = 'kullback-leibler'
save_fig = False
exp_name = '2wide'

In [ ]:
dataDir = 'data'
fullDir = os.path.join(dataDir, 'JammerData.mat')
data = io.loadmat(fullDir)
data = np.squeeze(data['data'])

fullDir = os.path.join(dataDir, 'filteredData.mat')
filteredData = io.loadmat(fullDir)
filteredData = np.squeeze(filteredData['data'])

In [ ]:
np.iscomplex(data).any()

In [ ]:
def plotSpectrogram(f, t, Sxx, vmin=None, vmax=None, save_fig=False, fig_path=None, figName=None):
    specdB = 10*np.log10(np.abs(Sxx))    
    cmap = plt.get_cmap('jet')
    
    if vmin==None and vmax==None:
        vmin = np.min(specdB)
        vmax = np.max(specdB)       
        
    plt.pcolormesh(t*1e3, f/1e6, specdB, vmin=vmin, vmax=vmax, cmap=cmap)
    plt.ylabel('Frequency [MHz]')
    plt.xlabel('Time [ms]')
    plt.colorbar().ax.set_ylabel('[dB]')
    plt.tight_layout()
    if save_fig:
        if not os.path.exists(fig_path):
            os.makedirs(fig_path)
        figName = ''.join([figName, '.eps'])
        plt.savefig(os.path.join(fig_path,figName), dpi=100, format='eps')
    plt.show()

In [ ]:
def saveFig(fig, fig_path, fig_name, save_fig=False):
    if save_fig:
        if not os.path.exists(fig_path):
            os.makedirs(fig_path)
        fig_name = ''.join([fig_name, '.eps'])
        fig.savefig(os.path.join(fig_path, fig_name), dpi=100, format='eps')

In [ ]:
f, t, Zxx = stft(data, fs=fs, nperseg=nperseg, nfft=nfft, boundary=None)
inputNMF = np.abs(Zxx)
inputNMF = np.where(inputNMF==0, 1e-12, inputNMF)

specdB = 10*np.log10(inputNMF)    
cmap = plt.get_cmap('jet')
vmin = np.min(specdB)
vmax = np.max(specdB)
fig = plt.figure()
ax = fig.add_subplot(111)
v = ax.pcolormesh(t*1e3, f/1e6, specdB, vmin=vmin, vmax=vmax, cmap=cmap)
ax.set_ylabel('Frequency [MHz]')
ax.set_xlabel('Time [ms]')
fig.colorbar(v).ax.set_ylabel('[dB]')
plt.tight_layout()
saveFig(fig, figPath, ''.join(['spec_input_', exp_name]), save_fig=save_fig)
plt.show()

In [ ]:
f, t, Zxx = stft(filteredData, fs=fs, nperseg=nperseg, nfft=nfft, boundary=None)
inputNMF = np.abs(Zxx)
inputNMF = np.where(inputNMF==0, 1e-12, inputNMF)

specdB = 10*np.log10(inputNMF)    
cmap = plt.get_cmap('jet')
fig = plt.figure()
ax = fig.add_subplot(111)
v = ax.pcolormesh(t*1e3, f/1e6, specdB, vmin=vmin, vmax=vmax, cmap=cmap)
ax.set_ylabel('Frequency [MHz]')
ax.set_xlabel('Time [ms]')
fig.colorbar(v).ax.set_ylabel('[dB]')
plt.tight_layout()
saveFig(fig, figPath, ''.join(['spec_input_', exp_name]), save_fig=save_fig)
plt.show()